In [8]:
def filter_data(df):
    tag_mapping = {
    'B-PERS': 'B-PER', 'I-PERS': 'I-PER',
    'B-PERS': 'B-PER', 'I-PERS': 'I-PER'
    }
    df = df[df['labels']!=-100].copy()
    df['true_labels'] = df['true_labels'].replace(tag_mapping)
    return df

def calculate_tokens_and_uniques(df):
    total_tokens = len(df)
    unique_tokens = len(df['core_tokens'].unique())
    return total_tokens, unique_tokens

def extract_tag_token_type(df):
    df = df[df['true_labels']!='O']
    tag_stats = df.groupby('true_labels').agg({
    'token_ids': ['count', 'nunique']
	}).reset_index()
    tag_stats.columns = ['Tag', 'Total Tokens', 'Tag Types']
    tag_stats['TTR'] = round(tag_stats['Tag Types'] / tag_stats['Total Tokens'], 4)
    return tag_stats


def extract_overlap_matrix(data):
    # Extracting unique types for each tag
    tags = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']
    tag_types = {tag: set() for tag in tags}
    
    for token, tag in zip(data['core_tokens'], data['true_labels']):
        normalised_tag = tag.replace('PERS', 'PER')
        if normalised_tag in tag_types:
            tag_types[normalised_tag].add(token)
    # Create an empty DataFrame for the overlap matrix
    overlap_matrix = pd.DataFrame(index=tags, columns=tags, dtype=int)
    for tag1 in tags:
            for tag2 in tags:
                if tag1 == tag2:
                    # Ignore diagonal by setting it to zero
                    overlap_matrix.loc[tag1, tag2] = 0
                else:
                    # Intersection count for tag pairs
                    overlap_matrix.loc[tag1, tag2] = len(tag_types[tag1].intersection(tag_types[tag2]))
    # overlap_matrix = np.log(overlap_matrix.replace(0, 0.01))
    return overlap_matrix


def extract_O_overlap_matrix(data):
    # Extracting unique types for each tag
    tags = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC', 'O']
    tag_types = {tag: set() for tag in tags}
    
    for token, tag in zip(data['core_tokens'], data['true_labels']):
        normalised_tag = tag.replace('PERS', 'PER')
        if normalised_tag in tag_types:
            tag_types[normalised_tag].add(token)
    # Create an empty DataFrame for the overlap matrix
    overlap_matrix = pd.DataFrame(index=tags, columns=tags, dtype=int)
    for tag1 in tags:
            for tag2 in tags:
                if tag1 == tag2:
                    # Ignore diagonal by setting it to zero
                    overlap_matrix.loc[tag1, tag2] = 0
                else:
                    # Intersection count for tag pairs
                    overlap_matrix.loc[tag1, tag2] = len(tag_types[tag1].intersection(tag_types[tag2]))
    # overlap_matrix = np.log(overlap_matrix.replace(0, 0.01))
    return overlap_matrix


def type_distribution(corpora, split):
	datasets_of_interest = ['ANERCorp', 'CoNLL-2003']
	tags_of_interest = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']

	# Prepare data for DataFrame
	all_data = []

	# Process each dataset
	for dataset in datasets_of_interest:
		data = corpora[dataset]['splits'][split]
		tag_word_counts = {tag: Counter() for tag in tags_of_interest}
		
		# Fill the dictionary with counts
		for tag in tags_of_interest:
			tag_word_counts[tag].update(w for w, t in zip(data['token_ids'], data['true_labels']) if t.replace('PERS', 'PER') == tag)

		# Calculate statistical measures for each tag
		for tag, counter in tag_word_counts.items():
			frequencies = np.array(list(counter.values()))
			if frequencies.size > 0:  # Check if there are any frequencies to calculate stats
				mean_freq = round(np.mean(frequencies), 4)
				median_freq = round(np.median(frequencies), 4)
				std_dev = round(np.std(frequencies), 2)
			else:
				mean_freq, median_freq, std_dev = 0, 0, 0

			all_data.append({
				'Dataset': dataset,
				'Tag': tag,
				'Mean Frequency': mean_freq,
				'Median Frequency': median_freq,
				'Standard Deviation': std_dev
			})
		 # Create DataFrame
		df = pd.DataFrame(all_data)

		# Return the DataFrame
	return df


def calculate_oov_rates_per_tag(corpora):
    results = {}

    for corpus_name, corpus_data in corpora.items():
        if corpus_name != 'ANERCorp_CamelLab-validation':
            if 'train' in corpus_data['splits'] and 'test' in corpus_data['splits']:
                train_words = {}
                test_words = {}
                
                # Collect words and their associated tags from training set
                for sentence in corpus_data['splits']['train']:
                    for word, tag in zip(sentence['words'], sentence['tags']):
                        normalised_tag = tag.replace('PERS', 'PER')
                        if normalised_tag not in train_words:
                            train_words[normalised_tag] = set()
                        train_words[normalised_tag].add(word)
                
                # Collect words and their associated tags from test set
                for sentence in corpus_data['splits']['test']:
                    for word, tag in zip(sentence['words'], sentence['tags']):
                        normalised_tag = tag.replace('PERS', 'PER')
                        if normalised_tag not in test_words:
                            test_words[normalised_tag] = set()
                        test_words[normalised_tag].add(word)
                
                # Calculate OOV rate per tag
                oov_stats = {}
                for tag in test_words:
                    if tag != 'O':
                        oov_words = test_words[tag] - train_words.get(tag, set())
                        oov_rate = len(oov_words) / len(test_words[tag]) if test_words[tag] else 0
                        oov_stats[tag] = {
                            'OOV Words Count': len(oov_words),
                            'Total Unique Words in Test': len(test_words[tag]),
                            'OOV Rate': round(oov_rate, 4)
                        }
                
                results[corpus_name] = oov_stats
            else:
                results[corpus_name] = "Missing 'train' or 'test' split data"
                
    return results

# Function to calculate tokenization metrics
def calculate_tokenization_metrics(df):
    df = df[df['true_labels'] != 'O'].copy()
    # Calculate tokenization rate
    df['tokenization_rate'] = df['word_pieces'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
    # Group by true_labels and calculate metrics
    tag_metrics = df.groupby('true_labels')['tokenization_rate'].agg(['mean', 'std', 'sum', 'count']).reset_index()
    tag_metrics.rename(columns={'sum': 'total_subwords', 'count': 'total_words'}, inplace=True)
    
    return tag_metrics






In [10]:
from pathlib import Path

In [11]:
anercorp_path = Path('/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02/extractions/analysis')
conoll_path = Path('/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/conll2003_bert/extractions/analysis')

In [12]:
anercorp_te = pd.read_json(
	anercorp_path / 'analysis_data.json',
	lines=True
)
print(f'ANERCorp:{len(anercorp_te)}')
aner_te = filter_data(anercorp_te)
print(f'ANERCorp:{len(aner_te)}')
anercorp_tr = pd.read_json(
	anercorp_path / 'train_data.json',
	lines=True
)
print(f'ANERCorp:{len(anercorp_tr)}')
aner_tr = filter_data(anercorp_tr)
print(f'ANERCorp:{len(aner_tr)}')
conll_te = pd.read_json(
	conoll_path / 'analysis_data.json',
	lines=True
)
print(f'conll:{len(conll_te)}')
con_te= filter_data(conll_te)
print(f'conll:{len(con_te)}')
conll_tr = pd.read_json(
	conoll_path / 'train_data.json',
	lines=True
)
print(f'conll:{len(conll_tr)}')
con_tr = filter_data(conll_tr)
print(f'conll:{len(con_tr)}')

ANERCorp:29711
ANERCorp:24991
ANERCorp:147082
ANERCorp:124659
conll:70367
conll:46435
conll:300677
conll:203621


In [13]:
corpora = {
	'ANERCorp': {
		'splits': {
			'train':aner_tr,
   			'test':aner_te
		}
	},
	'CoNLL-2003': {
		'splits': {
			'train':con_tr,
   			'test':con_te
		}
  
	}
}

In [14]:
# Applying the function to each dataset split
results = {
    "Dataset": [
        "ANERCorp (Train)", 
        "ANERCorp (Test)", 
        "CoNLL-2003 (Train)", 
        "CoNLL-2003 (Test)"
    ],
    "Total Tokens": [
        calculate_tokens_and_uniques(aner_tr)[0],
        calculate_tokens_and_uniques(aner_te)[0],
        calculate_tokens_and_uniques(con_tr)[0],
        calculate_tokens_and_uniques(con_te)[0]
    ],
    "Unique Words": [
        calculate_tokens_and_uniques(aner_tr)[1],
        calculate_tokens_and_uniques(aner_te)[1],
        calculate_tokens_and_uniques(con_tr)[1],
        calculate_tokens_and_uniques(con_te)[1]
    ]
}

# Create DataFrame
token_stats_df = pd.DataFrame(results)
token_stats_df

,Dataset,Total Tokens,Unique Words
0,ANERCorp (Train),124659,23010
1,ANERCorp (Test),24991,8455
2,CoNLL-2003 (Train),203621,11243
3,CoNLL-2003 (Test),46435,5883


In [15]:
# Calculate for each split
aner_tr_df = extract_tag_token_type(aner_tr)
aner_tr_df['Dataset'] = 'ANERCorp'
aner_tr_df['Split'] = 'Train'
conll_tr_df = extract_tag_token_type(con_tr)
conll_tr_df['Dataset'] = 'CoNLL-2003'
conll_tr_df['Split'] = 'Train'
aner_te_df = extract_tag_token_type(aner_te)
aner_te_df['Dataset'] = 'ANERCorp'
aner_te_df['Split'] = 'Test'
conll_te_df = extract_tag_token_type(con_te)
conll_te_df['Dataset'] = 'CoNLL-2003'
conll_te_df['Split'] = 'Test'

# Combine all results into a single DataFrame
train_split = pd.concat([aner_tr_df, conll_tr_df], ignore_index=True)
test_split = pd.concat([aner_te_df, conll_te_df], ignore_index=True)

In [16]:
aner_tr_df

,Tag,Total Tokens,Tag Types,TTR,Dataset,Split
0,B-LOC,3772,861,0.2283,ANERCorp,Train
1,B-MISC,887,333,0.3754,ANERCorp,Train
2,B-ORG,1576,507,0.3217,ANERCorp,Train
3,B-PER,2719,1003,0.3689,ANERCorp,Train
4,I-LOC,525,141,0.2686,ANERCorp,Train
5,I-MISC,375,217,0.5787,ANERCorp,Train
6,I-ORG,1115,391,0.3507,ANERCorp,Train
7,I-PER,2202,1005,0.4564,ANERCorp,Train


In [17]:

# Define your custom color map
color_map = {
    "B-LOC": "darkgreen",
    "B-PERS": "deepskyblue",
    "B-PER": "deepskyblue",
    "B-ORG": "darkcyan",
    "B-MISC": "palevioletred",
    "I-LOC": "yellowgreen",
    "I-PERS": "lightblue",
    "I-PER": "lightblue",
    "I-ORG": "cyan",
    "I-MISC": "violet",
    "O": "saddlebrown",
}

# Example data for ANERCorp and CoNLL-2003 training splits
# This would usually be filtered from a larger DataFrame as needed

anercorp_train = train_split[train_split['Dataset'] == 'ANERCorp']
conll_train = train_split[train_split['Dataset'] == 'CoNLL-2003']
anercorp_ttr = anercorp_train['Tag Types'].tolist()
anercorp_labels = anercorp_train['Tag'].tolist()
anercprp_colors = [color_map[tag] for tag in anercorp_labels]
conll_ttr = conll_train['Tag Types'].tolist()
conll_labels = conll_train['Tag'].tolist()
conll_colors = [color_map[tag] for tag in conll_labels]



# Create subplot for ANERCorp
fig_twr = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                             subplot_titles=("Words Proportion", "TTR"))
fig_twr.add_trace(go.Pie(labels=anercorp_labels, values=anercorp_ttr, name="TWR", marker_colors=anercprp_colors), 1, 1)
fig_twr.add_trace(go.Pie(labels=conll_labels, values=conll_ttr, name="TWR", marker_colors=conll_colors), 1, 2)

# Update layout for ANERCorp with a custom title
fig_twr.update_layout(
    title_text="Entity Tag Token Type Distribution in the Training Split",
    annotations=[
        dict(text='ANERCorp', font_size=20, showarrow=False),
        dict(text='CoNLL-2003',font_size=20, showarrow=False)

    ],
    height=500, width=800
)
fig_twr.show()


In [18]:

# Define your custom color map
color_map = {
    "B-LOC": "darkgreen",
    "B-PERS": "deepskyblue",
    "B-PER": "deepskyblue",
    "B-ORG": "darkcyan",
    "B-MISC": "palevioletred",
    "I-LOC": "yellowgreen",
    "I-PERS": "lightblue",
    "I-PER": "lightblue",
    "I-ORG": "cyan",
    "I-MISC": "violet",
    "O": "saddlebrown",
}

# Example data for ANERCorp and CoNLL-2003 training splits
# This would usually be filtered from a larger DataFrame as needed

anercorp_train = test_split[test_split['Dataset'] == 'ANERCorp']
conll_train = test_split[test_split['Dataset'] == 'CoNLL-2003']
anercorp_ttr = anercorp_train['Tag Types'].tolist()
anercorp_labels = anercorp_train['Tag'].tolist()
anercprp_colors = [color_map[tag] for tag in anercorp_labels]
conll_ttr = conll_train['Tag Types'].tolist()
conll_labels = conll_train['Tag'].tolist()
conll_colors = [color_map[tag] for tag in conll_labels]



# Create subplot for ANERCorp
fig_twr = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                             subplot_titles=("Words Proportion", "TTR"))
fig_twr.add_trace(go.Pie(labels=anercorp_labels, values=anercorp_ttr, name="TWR", marker_colors=anercprp_colors), 1, 1)
fig_twr.add_trace(go.Pie(labels=conll_labels, values=conll_ttr, name="TWR", marker_colors=conll_colors), 1, 2)

# Update layout for ANERCorp with a custom title
fig_twr.update_layout(
    title_text="Entity Tag Token Type Distribution in the Testing Split",
    annotations=[
        dict(text='ANERCorp', font_size=20, showarrow=False),
        dict(text='CoNLL-2003',font_size=20, showarrow=False)

    ],
    height=600, width=1000
)
fig_twr.show()


In [19]:
matrix_tags = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']

anercorp_overlap_matrix = extract_overlap_matrix(aner_tr)
conll_overlap_matrix = extract_overlap_matrix(con_tr)
# Convert dictionaries to DataFrames
matrix1 = pd.DataFrame(anercorp_overlap_matrix, index=matrix_tags)
matrix2 = pd.DataFrame(conll_overlap_matrix, index=matrix_tags)

# # # # Create masks for upper and lower triangles
# mask_upper = np.triu(np.ones_like(matrix1, dtype=bool))
# mask_lower = np.tril(np.ones_like(matrix1, dtype=bool))

# Create masks for lower triangles
mask_lower = np.tril(np.ones_like(matrix1, dtype=bool))

# Mask both matrices with the lower triangle
lower_triangle_anercorp = matrix1.mask(~mask_lower)
lower_triangle_conll = matrix2.mask(~mask_lower)

# Mask the appropriate halves
# upper_triangle = matrix1.mask(mask_upper)
# upper_triangle = matrix1
# lower_triangle = matrix2.mask(mask_lower)
# lower_triangle = matrix2

# Prepare text for display: replace NaN with blank and non-NaN with integer
upper_text = np.where(lower_triangle_anercorp.isnull(), '', lower_triangle_anercorp.fillna(0).astype(int).astype(str))
lower_text = np.where(lower_triangle_conll.isnull(), '', lower_triangle_conll.fillna(0).astype(int).astype(str))

# Create subplot with two heatmaps
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('ANERCorp', 'CoNLL-2003'),
    horizontal_spacing=0.1  # Adjust spacing between subplots
)

# Add the upper triangle of the first matrix
fig.add_trace(
    go.Heatmap(
        z=lower_triangle_anercorp,
        x=lower_triangle_anercorp.columns,
        y=lower_triangle_anercorp.index,
        colorscale="RdBu_r",
        showscale=False,  # Optionally turn off the color scale for each subplot
        text=upper_text.astype(str),
        texttemplate="%{text}",  # Text template to display
        hoverinfo="text+z"  # Display text and z-value on hover
    ),
    row=1, col=1
)

# Add the lower triangle of the second matrix
fig.add_trace(
    go.Heatmap(
        z=lower_triangle_conll,
        x=lower_triangle_conll.columns,
        y=lower_triangle_conll.index,
        colorscale="RdBu_r",
        showscale=True,  # Optionally show the color scale for reference
        text=lower_text.astype(str),
        texttemplate="%{text}",  # Text template to display
        hoverinfo="text+z"  # Display text and z-value on hover
    ),
    row=1, col=2
)

# Update the layout for better readability
fig.update_layout(
    title_text='Token Type Overlap Across Entity Tags in Training Split',
    template="plotly_white",
    xaxis_title="Tags",
    yaxis_title="Tags",
    xaxis=dict(showgrid=False),  # Hide gridlines for x-axis
    xaxis2=dict(showgrid=False), # Hide gridlines for x-axis in second subplot
    yaxis=dict(showgrid=False),  # Hide gridlines for y-axis
    yaxis2=dict(showgrid=False), # Hide gridlines for y-axis in second subplot
    height=600,  # Set height of the figure canvas in pixels
    width=1200   # Set width of the figure canvas in pixels
)

# Show the figure
fig.show()


In [20]:
matrix_O_tags = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC', 'O']

anercorp_overlap_matrix = extract_O_overlap_matrix(aner_tr)
conll_overlap_matrix = extract_O_overlap_matrix(con_tr)


# Convert dictionaries to DataFrames
matrix1 = pd.DataFrame(anercorp_overlap_matrix, index=matrix_O_tags)
matrix2 = pd.DataFrame(conll_overlap_matrix, index=matrix_O_tags)

# # # # Create masks for upper and lower triangles
# mask_upper = np.triu(np.ones_like(matrix1, dtype=bool))
# mask_lower = np.tril(np.ones_like(matrix1, dtype=bool))

# Create masks for lower triangles
mask_lower = np.tril(np.ones_like(matrix1, dtype=bool))

# Mask both matrices with the lower triangle
lower_triangle_anercorp = matrix1.mask(~mask_lower)
lower_triangle_conll = matrix2.mask(~mask_lower)

# Mask the appropriate halves
# upper_triangle = matrix1.mask(mask_upper)
# upper_triangle = matrix1
# lower_triangle = matrix2.mask(mask_lower)
# lower_triangle = matrix2

# Prepare text for display: replace NaN with blank and non-NaN with integer
upper_text = np.where(lower_triangle_anercorp.isnull(), '', lower_triangle_anercorp.fillna(0).astype(int).astype(str))
lower_text = np.where(lower_triangle_conll.isnull(), '', lower_triangle_conll.fillna(0).astype(int).astype(str))

# Create subplot with two heatmaps
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('ANERCorp', 'CoNLL-2003'),
    horizontal_spacing=0.1  # Adjust spacing between subplots
)

# Add the upper triangle of the first matrix
fig.add_trace(
    go.Heatmap(
        z=lower_triangle_anercorp,
        x=lower_triangle_anercorp.columns,
        y=lower_triangle_anercorp.index,
        colorscale="RdBu_r",
        showscale=False,  # Optionally turn off the color scale for each subplot
        text=upper_text.astype(str),
        texttemplate="%{text}",  # Text template to display
        hoverinfo="text+z"  # Display text and z-value on hover
    ),
    row=1, col=1
)

# Add the lower triangle of the second matrix
fig.add_trace(
    go.Heatmap(
        z=lower_triangle_conll,
        x=lower_triangle_conll.columns,
        y=lower_triangle_conll.index,
        colorscale="RdBu_r",
        showscale=True,  # Optionally show the color scale for reference
        text=lower_text.astype(str),
        texttemplate="%{text}",  # Text template to display
        hoverinfo="text+z"  # Display text and z-value on hover
    ),
    row=1, col=2
)

# Update the layout for better readability
fig.update_layout(
    title_text='Word Type Overlap Across Entity Tags in Training Split and O',
    template="plotly_white",
    xaxis_title="Tags",
    yaxis_title="Tags",
    xaxis=dict(showgrid=False),  # Hide gridlines for x-axis
    xaxis2=dict(showgrid=False), # Hide gridlines for x-axis in second subplot
    yaxis=dict(showgrid=False),  # Hide gridlines for y-axis
    yaxis2=dict(showgrid=False), # Hide gridlines for y-axis in second subplot
    height=600,  # Set height of the figure canvas in pixels
    width=1200   # Set width of the figure canvas in pixels
)

# Show the figure
fig.show()


In [21]:
matrix_tags = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']

anercorp_overlap_matrix = extract_overlap_matrix(aner_te)
conll_overlap_matrix = extract_overlap_matrix(con_te)
# Convert dictionaries to DataFrames
matrix1 = pd.DataFrame(anercorp_overlap_matrix, index=matrix_tags)
matrix2 = pd.DataFrame(conll_overlap_matrix, index=matrix_tags)

# # # # Create masks for upper and lower triangles
# mask_upper = np.triu(np.ones_like(matrix1, dtype=bool))
# mask_lower = np.tril(np.ones_like(matrix1, dtype=bool))

# Create masks for lower triangles
mask_lower = np.tril(np.ones_like(matrix1, dtype=bool))

# Mask both matrices with the lower triangle
lower_triangle_anercorp = matrix1.mask(~mask_lower)
lower_triangle_conll = matrix2.mask(~mask_lower)

# Mask the appropriate halves
# upper_triangle = matrix1.mask(mask_upper)
# upper_triangle = matrix1
# lower_triangle = matrix2.mask(mask_lower)
# lower_triangle = matrix2

# Prepare text for display: replace NaN with blank and non-NaN with integer
upper_text = np.where(lower_triangle_anercorp.isnull(), '', lower_triangle_anercorp.fillna(0).astype(int).astype(str))
lower_text = np.where(lower_triangle_conll.isnull(), '', lower_triangle_conll.fillna(0).astype(int).astype(str))

# Create subplot with two heatmaps
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('ANERCorp', 'CoNLL-2003'),
    horizontal_spacing=0.1  # Adjust spacing between subplots
)

# Add the upper triangle of the first matrix
fig.add_trace(
    go.Heatmap(
        z=lower_triangle_anercorp,
        x=lower_triangle_anercorp.columns,
        y=lower_triangle_anercorp.index,
        colorscale="RdBu_r",
        showscale=False,  # Optionally turn off the color scale for each subplot
        text=upper_text.astype(str),
        texttemplate="%{text}",  # Text template to display
        hoverinfo="text+z"  # Display text and z-value on hover
    ),
    row=1, col=1
)

# Add the lower triangle of the second matrix
fig.add_trace(
    go.Heatmap(
        z=lower_triangle_conll,
        x=lower_triangle_conll.columns,
        y=lower_triangle_conll.index,
        colorscale="RdBu_r",
        showscale=True,  # Optionally show the color scale for reference
        text=lower_text.astype(str),
        texttemplate="%{text}",  # Text template to display
        hoverinfo="text+z"  # Display text and z-value on hover
    ),
    row=1, col=2
)

# Update the layout for better readability
fig.update_layout(
    title_text='Token Type Overlap Across Entity Tags in Training Split',
    template="plotly_white",
    xaxis_title="Tags",
    yaxis_title="Tags",
    xaxis=dict(showgrid=False),  # Hide gridlines for x-axis
    xaxis2=dict(showgrid=False), # Hide gridlines for x-axis in second subplot
    yaxis=dict(showgrid=False),  # Hide gridlines for y-axis
    yaxis2=dict(showgrid=False), # Hide gridlines for y-axis in second subplot
    height=600,  # Set height of the figure canvas in pixels
    width=1200   # Set width of the figure canvas in pixels
)

# Show the figure
fig.show()


In [20]:
matrix_O_tags = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC', 'O']

anercorp_overlap_matrix = extract_O_overlap_matrix(aner_te)
conll_overlap_matrix = extract_O_overlap_matrix(con_te)


# Convert dictionaries to DataFrames
matrix1 = pd.DataFrame(anercorp_overlap_matrix, index=matrix_O_tags)
matrix2 = pd.DataFrame(conll_overlap_matrix, index=matrix_O_tags)

# # # # Create masks for upper and lower triangles
# mask_upper = np.triu(np.ones_like(matrix1, dtype=bool))
# mask_lower = np.tril(np.ones_like(matrix1, dtype=bool))

# Create masks for lower triangles
mask_lower = np.tril(np.ones_like(matrix1, dtype=bool))

# Mask both matrices with the lower triangle
lower_triangle_anercorp = matrix1.mask(~mask_lower)
lower_triangle_conll = matrix2.mask(~mask_lower)

# Mask the appropriate halves
# upper_triangle = matrix1.mask(mask_upper)
# upper_triangle = matrix1
# lower_triangle = matrix2.mask(mask_lower)
# lower_triangle = matrix2

# Prepare text for display: replace NaN with blank and non-NaN with integer
upper_text = np.where(lower_triangle_anercorp.isnull(), '', lower_triangle_anercorp.fillna(0).astype(int).astype(str))
lower_text = np.where(lower_triangle_conll.isnull(), '', lower_triangle_conll.fillna(0).astype(int).astype(str))

# Create subplot with two heatmaps
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('ANERCorp', 'CoNLL-2003'),
    horizontal_spacing=0.1  # Adjust spacing between subplots
)

# Add the upper triangle of the first matrix
fig.add_trace(
    go.Heatmap(
        z=lower_triangle_anercorp,
        x=lower_triangle_anercorp.columns,
        y=lower_triangle_anercorp.index,
        colorscale="RdBu_r",
        showscale=False,  # Optionally turn off the color scale for each subplot
        text=upper_text.astype(str),
        texttemplate="%{text}",  # Text template to display
        hoverinfo="text+z"  # Display text and z-value on hover
    ),
    row=1, col=1
)

# Add the lower triangle of the second matrix
fig.add_trace(
    go.Heatmap(
        z=lower_triangle_conll,
        x=lower_triangle_conll.columns,
        y=lower_triangle_conll.index,
        colorscale="RdBu_r",
        showscale=True,  # Optionally show the color scale for reference
        text=lower_text.astype(str),
        texttemplate="%{text}",  # Text template to display
        hoverinfo="text+z"  # Display text and z-value on hover
    ),
    row=1, col=2
)

# Update the layout for better readability
fig.update_layout(
    title_text='Word Type Overlap Across Entity Tags in Testing Split and O',
    template="plotly_white",
    xaxis_title="Tags",
    yaxis_title="Tags",
    xaxis=dict(showgrid=False),  # Hide gridlines for x-axis
    xaxis2=dict(showgrid=False), # Hide gridlines for x-axis in second subplot
    yaxis=dict(showgrid=False),  # Hide gridlines for y-axis
    yaxis2=dict(showgrid=False), # Hide gridlines for y-axis in second subplot
    height=600,  # Set height of the figure canvas in pixels
    width=1200   # Set width of the figure canvas in pixels
)

# Show the figure
fig.show()


In [32]:
# Creating the bar chart
fig = go.Figure()
tag_mapping = {
    'B-PERS': 'B-PER', 'I-PERS': 'I-PER',
    'B-PERS': 'B-PER', 'I-PERS': 'I-PER'
}
dataset_mapping = {
    'ANERCorp_CamelLab': 'ANERCorp'
}
train_split['Tag'] = train_split['Tag'].replace(tag_mapping)
train_split['Dataset'] = train_split['Dataset'].replace(dataset_mapping)
# Add bars for each dataset
for dataset in train_split['Dataset'].unique():
    subset = train_split[train_split['Dataset'] == dataset]
    fig.add_trace(go.Bar(
        x=subset['Tag'],
        y=subset['TTR'],
        name=dataset,
        text=subset['TTR'],
        textposition='auto'
    ))

# Update the layout
fig.update_layout(
    title='Type-to-Token Ratio (TTR) Across Entity Tags in ANERCorp and CoNLL-2003 Training Data',
    template="plotly_white",
    xaxis_title='Entity Tag',
    yaxis_title='TTR',
    barmode='group',
    legend_title="Dataset",
    height=500,
    width=1300
)

# Show the plot
fig.show()

In [33]:
train_split

,Tag,Total Tokens,Tag Types,TTR,Dataset,Split
0,B-LOC,3772,861,0.2283,ANERCorp,Train
1,B-MISC,887,333,0.3754,ANERCorp,Train
2,B-ORG,1576,507,0.3217,ANERCorp,Train
3,B-PER,2719,1003,0.3689,ANERCorp,Train
4,I-LOC,525,141,0.2686,ANERCorp,Train
5,I-MISC,375,217,0.5787,ANERCorp,Train
6,I-ORG,1115,391,0.3507,ANERCorp,Train
7,I-PER,2202,1005,0.4564,ANERCorp,Train
8,B-LOC,7140,754,0.1056,CoNLL-2003,Train
9,B-MISC,3438,494,0.1437,CoNLL-2003,Train


In [34]:
# Creating the bar chart
fig = go.Figure()
tag_mapping = {
    'B-PERS': 'B-PER', 'I-PERS': 'I-PER',
    'B-PERS': 'B-PER', 'I-PERS': 'I-PER'
}
dataset_mapping = {
    'ANERCorp_CamelLab': 'ANERCorp'
}
train_split['Tag'] = test_split['Tag'].replace(tag_mapping)
train_split['Dataset'] = test_split['Dataset'].replace(dataset_mapping)
# Add bars for each dataset
for dataset in test_split['Dataset'].unique():
    subset = test_split[test_split['Dataset'] == dataset]
    fig.add_trace(go.Bar(
        x=subset['Tag'],
        y=subset['TTR'],
        name=dataset,
        text=subset['TTR'],
        textposition='auto'
    ))

# Update the layout
fig.update_layout(
    title='Type-to-Token Ratio (TTR) Across Entity Tags in ANERCorp and CoNLL-2003 Testing Data',
    template="plotly_white",
    xaxis_title='Entity Tag',
    yaxis_title='TTR',
    barmode='group',
    legend_title="Dataset",
    height=500,
    width=1300
)

# Show the plot
fig.show()

In [35]:
train_split

,Tag,Total Tokens,Tag Types,TTR,Dataset,Split
0,B-LOC,3772,861,0.2283,ANERCorp,Train
1,B-MISC,887,333,0.3754,ANERCorp,Train
2,B-ORG,1576,507,0.3217,ANERCorp,Train
3,B-PER,2719,1003,0.3689,ANERCorp,Train
4,I-LOC,525,141,0.2686,ANERCorp,Train
5,I-MISC,375,217,0.5787,ANERCorp,Train
6,I-ORG,1115,391,0.3507,ANERCorp,Train
7,I-PER,2202,1005,0.4564,ANERCorp,Train
8,B-LOC,7140,754,0.1056,CoNLL-2003,Train
9,B-MISC,3438,494,0.1437,CoNLL-2003,Train


In [36]:
test_split

,Tag,Total Tokens,Tag Types,TTR,Dataset,Split
0,B-LOC,668,264,0.3952,ANERCorp,Test
1,B-MISC,235,120,0.5106,ANERCorp,Test
2,B-ORG,450,205,0.4556,ANERCorp,Test
3,B-PER,858,486,0.5664,ANERCorp,Test
4,I-LOC,83,34,0.4096,ANERCorp,Test
5,I-MISC,165,111,0.6727,ANERCorp,Test
6,I-ORG,275,151,0.5491,ANERCorp,Test
7,I-PER,641,400,0.6240,ANERCorp,Test
8,B-LOC,1668,351,0.2104,CoNLL-2003,Test
9,B-MISC,702,228,0.3248,CoNLL-2003,Test


In [37]:
type_distribution_df = type_distribution(corpora, 'train')
dataset_mapping = {
    'ANERCorp_CamelLab': 'ANERCorp'
}
type_distribution_df['Dataset'] = type_distribution_df['Dataset'].replace(dataset_mapping)
fig = px.bar(type_distribution_df, x='Tag', y='Standard Deviation', color='Dataset', text='Standard Deviation')
fig.update_layout(
    title_text="Standard Deviation of Token Type Frequencies Across Entity Tags in ANERCorp and CoNLL-2003 in Training Split",
    template="plotly_white",
    xaxis_title='Entity Tag',
    yaxis_title='STD',
    barmode='group',
    legend_title="Dataset",
    height=500,
    width=1300
)
fig.show()


In [38]:
type_distribution_df = type_distribution(corpora, 'test')
dataset_mapping = {
    'ANERCorp_CamelLab': 'ANERCorp'
}
type_distribution_df['Dataset'] = type_distribution_df['Dataset'].replace(dataset_mapping)
fig = px.bar(type_distribution_df, x='Tag', y='Standard Deviation', color='Dataset', text='Standard Deviation')
fig.update_layout(
    title_text="Standard Deviation of Word Type Frequencies Across Entity Tags in ANERCorp and CoNLL-2003 in Testing Split",
    template="plotly_white",
    xaxis_title='Entity Tag',
    yaxis_title='STD',
    barmode='group',
    legend_title="Dataset",
    height=500,
    width=1300
)
fig.show()


In [ ]:
aner_tr[aner_tr['true_labels'] == 'B-LOC']['token_ids'].value_counts().sort_values(ascending=False).head(50)

words
لبنان        173
العراق       111
الصين        100
إسرائيل       89
الولايات      76
إيران         62
فرنسا         61
مصر           54
هونج          50
الشرق         48
اسرائيل       46
مكة           46
روسيا         43
المغرب        43
لندن          40
بغداد         38
واشنطن        38
دبي           38
بريطانيا      37
بيروت         36
غزة           33
أفريقيا       33
نيويورك       30
ألمانيا       28
الخليج        28
بلنسية        26
دارفور        26
السودان       23
القاهرة       23
سوريا         22
دمشق          21
المدينة       20
قطر           20
أوروبا        20
مشهد          19
طهران         19
الدوحة        19
إيطاليا       18
باريس         18
الصومال       16
فلسطين        15
وبريطانيا     14
الإمارات      14
موسكو         14
أراجون        13
بكين          13
خراسان        13
إسبانيا       13
كوريا         13
البرازيل      13
Name: count, dtype: int64

In [ ]:
oov_stats_per_tag = calculate_oov_rates_per_tag(corpora)
data_frames = {}
for corpus, tags_data in oov_stats_per_tag.items():
    if isinstance(tags_data, dict):  # Ensure there's data to process
        df = pd.DataFrame.from_dict(tags_data, orient='index')
        data_frames[corpus] = df

# Creating a single DataFrame from the dictionary
all_data = pd.concat({
    k: v for k, v in data_frames.items()
}, axis=0)

# Rename the levels for clarity
all_data.index.names = ['Corpus', 'Tag']
all_data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Tags and their corresponding OOV rates for both datasets
COL = 'OOV Words Count'

anercorp_oov_rates = all_data.loc['ANERCorp'][COL].tolist()
anercorp_tags = all_data.loc['ANERCorp'].index.tolist()
anercorp_colors = [color_map[tag] for tag in anercorp_tags]
conll_oov_rates = all_data.loc['CoNLL-2003'][COL].tolist()
conll_tags = all_data.loc['CoNLL-2003'].index.tolist()
conll_colors = [color_map[tag] for tag in conll_tags]

# Create subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                    subplot_titles=['ANERCorp', 'CoNLL-2003'])

# Add pie charts to the subplot
fig.add_trace(go.Pie(labels=anercorp_tags, values=anercorp_oov_rates, name='ANERCorp', marker_colors=anercorp_colors), row=1, col=1)
fig.add_trace(go.Pie(labels=conll_tags, values=conll_oov_rates, name='CoNLL-2003', marker_colors=conll_colors), row=1, col=2)

# Update layout for clarity and better viewing
fig.update_layout(
    title_text='Out-of-Vocabulary (OOV) Token Distribution by Entity Type in ANERCorp and CoNLL-2003',
    # Adjust the layout to make the charts bigger
    width=1200, height=600
)

# Show the plot
fig.show()


In [68]:


# List to hold results from all datasets
results = []

# Loop through each dataset and each split
for corpus_name, corpus_data in corpora.items():
    for split_name, split_data in corpus_data['splits'].items():
        # Calculate metrics
        metrics = calculate_tokenization_metrics(split_data)
        metrics['Corpus'] = corpus_name
        metrics['Split'] = split_name
        results.append(metrics)

# Concatenate all results into a single DataFrame
tokenization_rate_df = pd.concat(results, ignore_index=True)

In [75]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming data is loaded into a DataFrame called df
# Create a simplified version of your data for demonstration
data = {
    'true_labels': ['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER']*4,
    'mean': [1.176, 1.280, 1.239, 1.319, 1.112, 1.250, 1.196, 1.341,
             1.164, 1.293, 1.377, 1.441, 1.096, 1.242, 1.123, 1.492,
             1.866, 1.730, 2.113, 1.860, 1.474, 1.704, 1.651, 2.280,
             1.798, 1.803, 2.121, 1.848, 1.618, 1.490, 1.567, 2.344],
    'std': [0.493, 0.581, 0.606, 0.595, 0.396, 0.486, 0.624, 0.624,
            0.498, 0.629, 0.786, 0.672, 0.401, 0.654, 0.381, 0.716,
            1.431, 1.250, 1.331, 1.051, 0.956, 1.273, 1.058, 1.099,
            1.350, 1.482, 1.388, 1.018, 1.054, 0.940, 0.995, 1.127],
    'Corpus': ['ANERCorp']*16 + ['CoNLL-2003']*16,
    'Split': ['train']*8 + ['test']*8 + ['train']*8 + ['test']*8
}

df = pd.DataFrame(data)

# Plot setup
fig = make_subplots(rows=1, cols=2, subplot_titles=("ANERCorp", "CoNLL-2003"))

# Add traces
for corpus in df['Corpus'].unique():
    corpus_data = df[df['Corpus'] == corpus]
    for split in ['train', 'test']:
        split_data = corpus_data[corpus_data['Split'] == split]
        trace = go.Bar(
            x=split_data['true_labels'],
            y=split_data['mean'],
            name=f"{split} - Mean",
            error_y=dict(type='data', array=split_data['std'], visible=True),
            marker=dict(line=dict(color='rgba(0, 0, 0, 1)', width=0.5)),
            opacity=0.6,
            offsetgroup=split,
            legendgroup=f"{corpus}-{split}"
        )
        col = 1 if corpus == 'ANERCorp' else 2
        fig.add_trace(trace, row=1, col=col)

# Update layout
fig.update_layout(
    title_text="Tokenization Rates Across Different Splits and Datasets",
    xaxis_title="Entity Type",
    yaxis_title="Average Tokenization Rate",
    legend_title="Dataset and Split",
    barmode='group',
    height=700,
    width=1200
)

fig.show()


In [77]:
import pandas as pd
import plotly.graph_objects as go

# Sample data based on your structure
data = {
    'true_labels': ['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER']*8,
    'mean': [1.176, 1.280, 1.239, 1.319, 1.112, 1.250, 1.196, 1.341,
             1.164, 1.293, 1.377, 1.441, 1.096, 1.242, 1.123, 1.492,
             1.866, 1.730, 2.113, 1.860, 1.474, 1.704, 1.651, 2.280,
             1.798, 1.803, 2.121, 1.848, 1.618, 1.490, 1.567, 2.344,
             1.176, 1.280, 1.239, 1.319, 1.112, 1.250, 1.196, 1.341,
             1.164, 1.293, 1.377, 1.441, 1.096, 1.242, 1.123, 1.492,
             1.866, 1.730, 2.113, 1.860, 1.474, 1.704, 1.651, 2.280,
             1.798, 1.803, 2.121, 1.848, 1.618, 1.490, 1.567, 2.344],
    'std': [0.493, 0.581, 0.606, 0.595, 0.396, 0.486, 0.624, 0.624,
            0.498, 0.629, 0.786, 0.672, 0.401, 0.654, 0.381, 0.716,
            1.431, 1.250, 1.331, 1.051, 0.956, 1.273, 1.058, 1.099,
            1.350, 1.482, 1.388, 1.018, 1.054, 0.940, 0.995, 1.127]*2,
    'Corpus': ['ANERCorp']*16 + ['CoNLL-2003']*16 + ['ANERCorp']*16 + ['CoNLL-2003']*16,
    'Split': ['train']*8 + ['test']*8 + ['train']*8 + ['test']*8 + ['train']*8 + ['test']*8 + ['train']*8 + ['test']*8
}

df = pd.DataFrame(data)

# Group by labels, split, and corpus, and then create a plot
fig = go.Figure()

for corpus in df['Corpus'].unique():
    for split in ['train', 'test']:
        subset = df[(df['Corpus'] == corpus) & (df['Split'] == split)]
        fig.add_trace(go.Bar(
            x=subset['true_labels'],
            y=subset['mean'],
            name=f"{corpus} {split}",
            error_y=dict(type='data', array=subset['std'], visible=True),
            marker=dict(line=dict(color='rgba(0, 0, 0, 1)', width=0.5)),
            opacity=0.6
        ))

fig.update_layout(
    barmode='group',
    template="plotly_white",
    title_text="Tokenization Rates Comparison Across Datasets and Splits",
    xaxis_title="Entity Type",
    yaxis_title="Average Tokenization Rate",
    legend_title="Dataset and Split",
    height=600,
    width=1200
)

fig.show()


In [78]:
import plotly.graph_objects as go

# Example data setup
labels = ['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER']
values_ANERCorp_train = [3, 2, 3.5, 4, 3.2, 2.8, 3.5, 4.2]  # Example values
values_ANERCorp_test = [2.8, 2.2, 3.2, 3.8, 3, 2.5, 3.3, 3.9]
values_CoNLL_train = [4, 3.5, 4.5, 5, 4.2, 3.8, 4.5, 5.2]
values_CoNLL_test = [3.8, 3.2, 4.2, 4.8, 3.9, 3.5, 4.3, 4.9]

fig = go.Figure()
fig.add_trace(go.Bar(x=labels, y=values_ANERCorp_train, name='ANERCorp train', marker_color='rgba(255, 99, 71, 0.6)'))
fig.add_trace(go.Bar(x=labels, y=values_ANERCorp_test, name='ANERCorp test', marker_color='rgba(100, 149, 237, 0.6)'))
fig.add_trace(go.Bar(x=labels, y=values_CoNLL_train, name='CoNLL-2003 train', marker_color='rgba(75, 0, 130, 0.6)'))
fig.add_trace(go.Bar(x=labels, y=values_CoNLL_test, name='CoNLL-2003 test', marker_color='rgba(255, 215, 0, 0.6)'))

fig.update_layout(barmode='group', title='Tokenization Rates Comparison Across Datasets and Splits')
fig.show()
